In [101]:
from db.data import Data
import gurobipy as gp
from gurobipy import GRB
from scipy.special import comb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplleaflet

In [205]:
data = Data()

demand = data.get_demand()
waiting = data.get_waiting()
distances = data.get_distances()

RADIUS = 4000
MAX_OFFICERS = 1
ALPHA = 1
LAMBDA = 1.35
SERVICE_TIME = 1

In [206]:
# list(set(waiting['name']) - set(distances['WPname']))

In [207]:
distances['in_range'] = (distances['distance'] <= RADIUS).astype(int)
distances = distances.pivot(index='WPname',columns='DAuid',values='in_range')
# distances = distances.loc[:, (distances.sum() > 0)]
distances.drop('Holland Park', inplace=True)

In [208]:
DAuid = list(distances.columns)
demand = demand[demand['DAuid'].isin(DAuid)]

In [233]:
distances.head()

DAuid,59151838,59151839,59151840,59151841,59151842,59151843,59151844,59151845,59151846,59151847,...,59154008,59154009,59154010,59154011,59154020,59154021,59154022,59154023,59154024,59154025
WPname,,,,,,,,,,,,,,,,,,,,,
APH Matthew Park,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Aria Banquet Convention Center,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bear Creek Park,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Blumsen Park,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,1,0,1,0
Bolivar Park,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [210]:
coverage = {}
for index, row in distances.iterrows():
    coverage[index] = [set(row[row==1].keys())]
    
pop_dict = pd.Series(demand.population_val.values,index=(demand.DAuid)).to_dict()

In [211]:
crime = data.get_crime()
crime = crime[crime['distance'] < 200] #removing errors
crime = demand.set_index('DAuid').join(crime['closest_demand'].value_counts()).reset_index().rename(columns={'closest_demand':'incidents'})
crime['incidents'] = crime['incidents'].replace(np.nan, 0)
crime.head()

,DAuid,population_val,DAarea,lat,lon,FEDcode,FEDName,incidents
0,59151838,941.0,2.5917,49.187082,-122.740379,12,Fleetwood/Port Kells,16.0
1,59151839,303.0,2.7945,49.169710,-122.745703,12,Fleetwood/Port Kells,12.0
2,59151840,801.0,0.7341,49.173207,-122.765197,12,Fleetwood/Port Kells,20.0
3,59151841,731.0,0.2013,49.175556,-122.772461,12,Fleetwood/Port Kells,20.0
4,59151842,881.0,0.3040,49.172899,-122.775759,12,Fleetwood/Port Kells,8.0


In [212]:
MAX_OFFICERS=20
ALPHA=1
regions, population = gp.multidict(pop_dict)
waits, covered = gp.multidict(coverage)

prob = [comb(MAX_OFFICERS,k)*((0.6)**k)*((0.4)**(MAX_OFFICERS-k)) for k in range(0,MAX_OFFICERS+1)]

In [213]:
m = gp.Model("SPMARP")
m.Params.LogToConsole = 0 #dont print model information

In [214]:
wait_size = [(name,size) for name in list(waiting['name']) for size in range(0,MAX_OFFICERS+1)]
demand_size = [(DAuid,size) for DAuid in list(distances.columns) for size in range(0,MAX_OFFICERS+1)]

In [215]:
#variable about where to place officers
build = m.addVars(wait_size, vtype=GRB.BINARY, name="wait_points")
is_covered = m.addVars(demand_size, vtype=GRB.BINARY, name="Is_covered")
build_change = m.addVars(wait_size, vtype=GRB.BINARY, name="wait_change")

In [216]:
#sum demand points only where officers can reach
for size in range(0,MAX_OFFICERS+1):
    m.addConstrs((gp.quicksum(build[wait,size] for wait in waits if r in covered[wait]) >= is_covered[r,size]
                            for r in regions), name="Build2cover")

In [217]:
#only allow max officers
for size in range(0,MAX_OFFICERS+1):
    m.addConstr(gp.quicksum(build[wait,size] for wait in waits) == size, name="officers")

In [218]:
#Control waiting site change
for size in range(1,MAX_OFFICERS):
    m.addConstrs( (build[wait,size] - build[wait,size+1] <= build_change[wait,size] for wait in waits), name="cease_wait")

In [219]:
#Max amount of location changes
for size in range(1,MAX_OFFICERS):
    m.addConstrs( (build_change[wait,size] <= ALPHA for wait in waits), name="cease_wait")

In [220]:
obj = gp.quicksum(prob[k]*population[r]*is_covered[r,k] for r in regions for k in range(1,MAX_OFFICERS+1))

In [221]:
m.setObjective(obj, GRB.MAXIMIZE)

In [222]:
m.optimize()

In [223]:
WPnames = []
for tower in build.keys():
    if (abs(build[tower].x) > 1e-6):
        if tower[1] == MAX_OFFICERS:
            WPnames.append(tower[0])
            print(f"\n Place officer at location {tower}.")
WPnames = list(set(WPnames))


 Place officer at location ('LA Matheson Secondary School/Moffat Park', 20).

 Place officer at location ('Scott Road Skytrain', 20).

 Place officer at location ('Scott Road Centre Shopping Mall', 20).

 Place officer at location ('Sullivan Park', 20).

 Place officer at location ('Panorama Park', 20).

 Place officer at location ("King's Cross Shopping Centre", 20).

 Place officer at location ('Fraser Heights Park', 20).

 Place officer at location ('Enver Creek Park', 20).

 Place officer at location ('Riverside Heights Shopping Centre Shopping Mall', 20).

 Place officer at location ('Surrey Bend Regional Park', 20).

 Place officer at location ('RCMP E-Division Headquarters', 20).

 Place officer at location ('Bonnie Schrenk Park', 20).

 Place officer at location ('Hazelgrove Park', 20).

 Place officer at location ('Royal Canadian Mounted Police Cloverdale', 20).

 Place officer at location ('Crescent Park', 20).

 Place officer at location ('Peace Arch Provincial Park', 20).


In [122]:
waiting_plot = waiting[waiting['name'].isin( WPnames )]
    # plt.plot(demand['lon'], demand['lat'], 'rs')
plt.plot(waiting_plot['lon'].astype(np.float64), waiting_plot['lat'].astype(np.float64), 'bs')
mplleaflet.show()

In [225]:
#STATIC CASE
MAX_OFFICERS=20
m = gp.Model("SPMARP")
m.Params.LogToConsole = 0 #dont print model information
#variable about where to place officers
build = m.addVars(list(waiting['name']), vtype=GRB.BINARY, name="wait_points")
is_covered = m.addVars(list(distances.columns), vtype=GRB.BINARY, name="Is_covered")

#sum demand points only where officers placed
m.addConstrs((gp.quicksum(build[t] for t in waits if r in covered[t]) >= is_covered[r]
                        for r in regions), name="Build2cover")

#only allow max officers
m.addConstr(gp.quicksum(build[t] for t in waits) == MAX_OFFICERS, name="officers")

m.setObjective(gp.quicksum(is_covered[r]*population[r] for r in regions), GRB.MAXIMIZE)

m.optimize()

print(f'model size={MAX_OFFICERS}')
WPnames2 = []
for tower in build.keys():
    if (abs(build[tower].x) > 1e-6):
        WPnames2.append(tower)
        print(f"\n Place officer at location {tower}.")
WPnames2 = list(set(WPnames))

total_population = 0

for region in regions:
    total_population += population[region]

percent = round(100*m.objVal/total_population, 2)

print(f"\n The population coverage associated to the patrol officer distribution is: {percent} %\n\n")   

model size=20

 Place officer at location Bolivar Park.

 Place officer at location Scott Road Centre Shopping Mall.

 Place officer at location Sullivan Park.

 Place officer at location Hazelnut Meadows Community Park.

 Place officer at location Panorama Park.

 Place officer at location Robson Park.

 Place officer at location Fraser Heights Park.

 Place officer at location Riverside Heights Shopping Centre Shopping Mall.

 Place officer at location Surrey Bend Regional Park.

 Place officer at location Maple Green Park.

 Place officer at location Bonnie Schrenk Park.

 Place officer at location Port Kells Park.

 Place officer at location Royal Canadian Mounted Police Cloverdale.

 Place officer at location Hillcrest Park.

 Place officer at location Peace Arch Provincial Park.

 Place officer at location Redwood Park.

 Place officer at location Dr. RJ Allan Hogg Rotary Park.

 Place officer at location Dogwood Park.

 Place officer at location Blumsen Park.

 Place officer at 

In [194]:
waiting_plot = waiting[waiting['name'].isin( WPnames2 )]
    # plt.plot(demand['lon'], demand['lat'], 'rs')
plt.plot(waiting_plot['lon'].astype(np.float64), waiting_plot['lat'].astype(np.float64), 'bs')
mplleaflet.show()